importing necessary modules

In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait

setting up chrome settings:hheadless+window size

In [2]:
options=Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--window-size=1920,1200')

getting our site

In [3]:
driver=webdriver.Chrome(options=options)
site='https://www.jumia.co.ke/'
driver.get(site)

clicking on news pop up

In [4]:
news=driver.find_element(By.CLASS_NAME,"cls")
news.click()

clicking on accept cookies pop up

In [5]:
cookies=driver.find_element(By.CLASS_NAME,"banner-pop").find_element(By.TAG_NAME,"button").click()

clicking on phones option where we want to scrape some phone data

In [6]:
phones=driver.find_element(By.CLASS_NAME,"flyout").find_elements(By.CLASS_NAME,'itm')
phones[1].click()

checking the different phone categories available

In [7]:
categories=driver.find_element(By.ID,"ctlg").find_elements(By.CLASS_NAME,"col16")
len(categories)

9

In [8]:
data=[]
for category in categories[3:8]:
    #title of the category
    title=category.find_element(By.TAG_NAME,'header').find_element(By.TAG_NAME,'h2').text
    #secondary link for the category
    link=category.find_element(By.CLASS_NAME,'col').find_element(By.TAG_NAME,'a').get_attribute('href')
    #getting secondary link for the specific category
    driver1=webdriver.Chrome(options=options)
    driver1.get(link)
    #sleeping as we wait for the whole page to load
    time.sleep(10)
    #clicking on news and cookies pop ups
    news=driver1.find_element(By.CLASS_NAME,"cls")
    news.click()
    cookies=driver1.find_element(By.CLASS_NAME,"banner-pop").find_element(By.TAG_NAME,"button").click()
    print(link)
    driver.implicitly_wait(20)
    print('waiting')
    time.sleep(10)
    #soup object 
    soup=BeautifulSoup(driver1.page_source,'html.parser')
    print('finished')
    catalog=soup.find('div',class_='-phs -pvxs row _no-g _4cl-3cm-shs').find_all('article')
    #looping through each item extracting raw required data and storing it in a list
    for item in catalog:
        store=[]
        info=item.find('div',class_='info')
        spec=info.find('h3',class_='name').text
        specs=spec.split(',')
        name=specs[0]
        originalprice=info.find('div',class_='prc').text
        try:
            rating=info.find('div',class_='rev').text
            discountedprice=info.find('div',class_='s-prc-w').find('div',class_='old').text
            discount=info.find('div',class_='s-prc-w').find('div',class_='bdg _dsct _sm').text
        except AttributeError:
            rating=' '
            discountedprice=' '
            discount=' '
        store=[title,name,specs,originalprice,discountedprice,discount,rating]
        data.append(store)
    driver1.quit()

https://www.jumia.co.ke/mlp-tecno-official-store/
waiting
finished
https://www.jumia.co.ke/mlp-oppo-store/
waiting
finished
https://www.jumia.co.ke/cell-phones/
waiting
finished
https://www.jumia.co.ke/smartphones/?tag=REFU
waiting
finished
https://www.jumia.co.ke/mobile-accessories
waiting
finished


In [9]:
driver.quit()

In [11]:
data[0]

['Deals From Tecno',
 'Tecno CAMON 30S',
 ['Tecno CAMON 30S',
  ' 6.78" FHD+AMOLED 120Hz',
  ' 128GB ROM+6GB RAM (+6GB Extended RAM) (4G Dual Sim)',
  ' 5000mAh- Celestial Black (1YR WRTY)'],
 'KSh 19,999',
 'KSh 24,999',
 '20%',
 '4.1 out of 5(16)']

storing the data in a dataframe

In [12]:
columns=['BRAND','NAME','SPECS','D.PRICE','O.PRICE','DSCT','RATING']
df=pd.DataFrame(data)
df.columns=columns
df.head()

,BRAND,NAME,SPECS,D.PRICE,O.PRICE,DSCT,RATING
0,Deals From Tecno,Tecno CAMON 30S,"[Tecno CAMON 30S, 6.78"" FHD+AMOLED 120Hz, 12...","KSh 19,999","KSh 24,999",20%,4.1 out of 5(16)
1,Deals From Tecno,"Tecno POP 9 120Hz 6.67"" 64GB ROM+3GB RAM*(+3GB...","[Tecno POP 9 120Hz 6.67"" 64GB ROM+3GB RAM*(+3G...","KSh 12,999",,,
2,Deals From Tecno,Tecno Spark 30c,"[Tecno Spark 30c, 6.67'' HD+, UP To 128GB RO...","KSh 14,999",,,
3,Deals From Tecno,Tecno POP 8,"[Tecno POP 8, 6.6"", 2GB RAM + 64GB, 13MP, ...","KSh 8,999","KSh 10,999",18%,4.4 out of 5(73)
4,Deals From Tecno,Tecno CAMON 30 Premier 5G 6.77'' AMOLED Screen,[Tecno CAMON 30 Premier 5G 6.77'' AMOLED Scree...,"KSh 41,999","KSh 74,999",44%,4.6 out of 5(5)
